In [ ]:
import numpy as np
import pandas as pd
        

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
plt.style.use('ggplot')
import dalex as dx

from scipy import stats
from scipy.stats import norm


from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer 

import math
import warnings
warnings.filterwarnings('ignore')
np.random.seed(23)

In [ ]:
full_df = pd.read_csv('BankChurners.csv')

In [ ]:
# We don't need the unique ids'
full_df.drop('CLIENTNUM', axis=1, inplace=True)

full_df.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1'], 
          inplace=True, axis=1)
full_df.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'], 
          inplace=True, axis=1)

display(full_df.shape)

In [ ]:
updated_df = pd.DataFrame()

def tobinary():
    
    # full_df['Attrition_Flag'] = full_df.Attrition_Flag // same thing
    updated_df['Attrition'] = full_df.Attrition_Flag.map({'Existing Customer':1, 'Attrited Customer':0})
    
    updated_df['Gender'] = full_df.Gender.map({'M':1, 'F':0})

In [ ]:
numeric_columns = ['Customer_Age','Credit_Limit','Months_on_book','Avg_Utilization_Ratio','Total_Trans_Amt','Dependent_count',
                  'Total_Relationship_Count','Months_Inactive_12_mon','Contacts_Count_12_mon','Total_Revolving_Bal',
                  'Total_Amt_Chng_Q4_Q1','Total_Ct_Chng_Q4_Q1']

def stringtoint():
    missing_income = full_df['Income_Category'].replace({'Unknown': 1 , 'Less than $40K':0, '$40K - $60K':0, 
                                                      '$80K - $120K':0, '$60K - $80K':0, '$120K +':0})
    #missinng data will be replaced with mode:
    income_data    = full_df['Income_Category'].replace({'Unknown': 1 , 'Less than $40K':1, '$40K - $60K':2, 
                                                      '$80K - $120K':4, '$60K - $80K':3, '$120K +':5})
    
    
    
    missing_education = full_df['Education_Level'].replace({'Unknown': 1, 'High School':0, 'Graduate':0, 'Uneducated':0,
                                                         'College':0,'Post-Graduate':0,'Doctorate':0})
    #missinng data will be replaced with mode:
    education_data    = full_df['Education_Level'].replace({'Unknown': 4, 'High School':2, 'Graduate':4, 'Uneducated':1,
                                                         'College':3,'Post-Graduate':5,'Doctorate':6})
    
    updated_df['Missing_Income']    = missing_income
    updated_df['Income_Category']   = income_data
    updated_df['Missing_Education'] = missing_education
    updated_df['Education_Level']   = education_data

In [ ]:
def encode():
    global updated_df
    card_dummies = pd.get_dummies(full_df['Card_Category'], prefix='Card')
    marital_dummies = pd.get_dummies(full_df['Marital_Status'], prefix='Marital')
    updated_df = pd.concat([updated_df, marital_dummies, card_dummies], axis=1)

In [ ]:
def concat_with_numerics():
    global updated_df
    updated_df = pd.concat([updated_df, full_df.loc[:, numeric_columns]], axis=1)

In [ ]:
tobinary()
stringtoint()
encode()
concat_with_numerics()

In [ ]:
import pickle

In [ ]:
m_logreg = LogisticRegression()

# m_logreg = pickle.load(open("m_logreg", 'rb'))

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(updated_df.drop('Attrition', axis = 1), updated_df.Attrition, test_size=0.2)

In [ ]:
m_logreg = m_logreg.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error

y_pred = m_logreg.predict(X_test)
mean_squared_error(y_test, y_pred, squared = False)

In [ ]:
from sklearn import metrics
metrics.plot_roc_curve(m_logreg, X_test, y_test)  
plt.rcParams['figure.figsize'] = [10, 10]
plt.show()

# Dla wybranej obserwacji ze zbioru danych wylicz predykcję modelu

In [ ]:
explainer = dx.Explainer(m_logreg, X_train, y_train)

In [ ]:
explainer.predict(X_train)[4]

Do zadania został użyty model regresji logicznej. Przglądając różne obserwację otrzymywałam takie wyniki jak: 0,34; 0,76; 0,98. My przyjrzymy się obserwacji o indexie 4, której predykcja wynosi ok.0.84.

# Dla wybranej obserwacji z punktu 1., wylicz dekompozycję predykcji modelu

In [ ]:
pp = explainer.predict_parts(X_train.iloc[4,:], type='shap')
pp.plot()

Dla obserwacji o indexie 4 największą udział w predykcji mają zmienne:
- `Total_Relationship_Count` = 6, klient korzysta intensywnie z usług, więc nie ma zamiaru zamykać konta
- `Total_Revolving_Bal` = 0, klient ma bardzo ograniczone możliwości korzystania z karty, więc ujemny udział zmiennej jest może być uzasadniony
- `Contacts_Count_12_mon` = 4, klient nie zrezygnuje z usługi, ponieważ potrzebuję konta do prowadzenia tranzakcji ze swoimi "kontaktami"


# Wybierz dwie obserwacje ze zbioru danych, które mają inne najważniejsze zmienne

In [ ]:
pp_shap_C = explainer.predict_parts(X_train.iloc[21,:], type='shap')
pp_shap_C.plot()
pp_shap_D = explainer.predict_parts(X_train.iloc[1,:], type='shap')
pp_shap_D.plot()

Pierwszy:
- `Total Revolving Bal` -0.141
- `Moths Inactive 12 mon` 0.061


Drugi:
- `Total Relationship Count`-0.12
- `Total Trans Amt` 0.069

Dla pierwszej obserwacji zmienna `Total_Revolving_Bal` ma dużo większy udział niż dla drugiej. Klienci posiadający saldo zerowe nie mogą korzystać z usługi, co może być dużą motywacją do rezygnacji z konta. Wsród zmienych mający często duży udział w predykcji, największym udział dla konkretnej obserwacji mają zmienne, które przyjmują wartości brzegowe/ odchylone od średniej.


In [ ]:
pp_shap = explainer.predict_parts(X_train.iloc[26,:], type='shap')
pp_shap.plot()
pp_shap = explainer.predict_parts(X_train.iloc[30,:], type='shap')
pp_shap.plot()

Tak jak wcześniej zauważyłam, zmienne ktróe przyjmują swoje wartości brzegowe jak np. `Total_Relationship_Count`, mają dużą kontrybucję. Dodatkowo dla tej zmiennej można by przypuszczać że, to czy kontrybucja jest dodatnia lub ujemna zależy od tego którą wartość brzegową przyjmiemy - 1 lub 5. Podobnie wygląda to dla zmiennej `Total_Trans_Amt`. Przyjrzyjmy się także zmiennej `Months_Inactive_12_mon`. W obu przypadkach przyjmuję ona wartość 3, a jej kontrybucja wynosi -0.018 w pierwszym i -0.2 w drugi.Są to wartości podobne, ale nie identyczne.